In [1]:
#- 패키지 임포트
import pandas as pd
from tqdm import tqdm
import re
from konlpy.tag import Okt
okt = Okt()
from collections import Counter # 형태소별 빈도 구할 때 사용
from IPython.display import clear_output
from IPython import display # 램효율을 늘리기 위해 아웃풋 display를 지워줄 것.
from ipywidgets import Output

out = Output()
display.display(out)

Output()

In [2]:
''' 파일 불러오기 '''
df = pd.read_csv("../dataset/2022/본문_전처리/나무문화_본문_10월_전처리.txt", encoding='UTF-8')
print(df.shape, df.columns)
corpus = df['split_str'].to_list() # 문장(X문서X)으로 가져와서 실행.
# ↑ 추후에 다른 분석 시도할 때 문장 별 형태소가 필요할 때도 있음. 문장별 형태소와 이를 문서로 합친 게 있으면 편하기 때문에 문장별로 형태소 분석 실행.
print(len(corpus))

(1113269, 2) Index(['split_str', 'org_idx'], dtype='object')
1113269


In [3]:
''' 형태소 분석 '''
nouns = []
verbs = []
adjs = []
josas = [] # 불용어 목록에 조사 사용하기 위함.
morphs = [] # 품사 태깅 없이 형태소만 추출. 추후 TF-IDF, N-gram등 텍스트 분석은 이 데이터로 진행함.
pos_list = [] # 감성분석할 때 필요.

In [4]:
for i in tqdm(range(len(corpus))):
    # 답변에서 형태소/품사 추출
    try:
        a = okt.pos(corpus[i], norm=True, stem=True) # 단어의 정규화와 어간 추출을 실행(True).
        m = okt.morphs(corpus[i], norm=True, stem=False)
        morphs.append(m) # 형태소 추가
        pos_list.append(a) # 형태소/품사 추가
        for x, y in a:
            # 품사가 명사면 명사 리스트에 단어 추가
            if y == 'Noun':
                nouns.append(x)
            # 품사가 동사면 동사 리스트에 단어 추가
            elif y == 'Verb':
                verbs.append(x)
            # 품사가 형용사면 형용사 리스트에 단어 추가
            elif y == 'Adjective':
                adjs.append(x)
		    # 품사가 조사면 조사 리스트에 단어 추가.
            elif y == 'Josa':
                josas.append(x)
    except:
        pass

100%|██████████| 1113269/1113269 [23:07:30<00:00, 13.37it/s]   


In [5]:
# 명사, 동사, 형용사 리스트에서 10개 단어씩 출력해서 확인
print(nouns[0:10])
print(verbs[0:10])
print(adjs[0:10])
print(josas[0:10])

['날', '캠핑', '포스팅', '깜빡', '중이', '날', '날씨', '캠핑장', '펜션', '꽉']
['가다', '미루다', '하다', '버리다', '몰다', '쓰다', '보다', '버리다', '얻다', '하다']
['좋다', '당연하다', '있다', '친하다', '차갑다', '많다', '차갑다', '굉장하다', '좋다', '있다']
['인데', '을', '다', '가', '과', '은', '로', '을', '와', '와']


In [6]:
''' 명사, 동사, 형용사, 조사 똑같이 아래 코드 반복 '''
# 명사 리스트에서 명사 빈도 리스트 생성
print(len(Counter(nouns))) # 이 값을 밑에 most_common 안에 넣으면 됨.
noun_cnt = Counter(nouns).most_common()

명사 = []
명사빈도 = []
for a, b in noun_cnt:
    명사.append(a)
    명사빈도.append(b)

# 단어와 빈도를 가지고 판다스 데이터프레임(엑셀 표와 비슷) 생성
noun_df = pd.DataFrame({'명사':명사, '명사빈도':명사빈도})
# 명사 데이터프레임을 파일로 저장. .xlsx, .csv, .txt 등 가능.
noun_df.to_csv("../dataset/2022/본문_형태소분석/10월/나무문화_10월_형태소분석(명사).csv", index=False, encoding='UTF-8')
noun_df

165188


,명사,명사빈도
0,것,230802
1,수,149908
2,그,119198
3,이,114732
4,곳,92033
...,...,...
165183,버지니아비치,1
165184,눙알,1
165185,트잉여,1
165186,번완당,1


In [7]:
''' 명사, 동사, 형용사, 조사 똑같이 아래 코드 반복 '''
# 명사 리스트에서 명사 빈도 리스트 생성
print(len(Counter(verbs))) # 이 값을 밑에 most_common 안에 넣으면 됨.
verb_cnt = Counter(verbs).most_common()

동사 = []
동사빈도 = []
for a, b in verb_cnt:
    동사.append(a)
    동사빈도.append(b)

# 단어와 빈도를 가지고 판다스 데이터프레임(엑셀 표와 비슷) 생성
verb_df = pd.DataFrame({'동사':동사, '동사빈도':동사빈도})
# 명사 데이터프레임을 파일로 저장. .xlsx, .csv, .txt 등 가능.
verb_df.to_csv("../dataset/2022/본문_형태소분석/10월/나무문화_10월_형태소분석(동사).csv", index=False, encoding='UTF-8')
verb_df

1880


,동사,동사빈도
0,하다,1025135
1,되다,208043
2,보다,127915
3,되어다,95344
4,않다,94652
...,...,...
1875,노닥대다,1
1876,찰칵대다,1
1877,할딱이다,1
1878,벋디디다,1


In [8]:
''' 명사, 동사, 형용사, 조사 똑같이 아래 코드 반복 '''
# 명사 리스트에서 명사 빈도 리스트 생성
print(len(Counter(adjs))) # 이 값을 밑에 most_common 안에 넣으면 됨.
adj_cnt = Counter(adjs).most_common()

형용사 = []
형용사빈도 = []
for a, b in adj_cnt:
    형용사.append(a)
    형용사빈도.append(b)

# 단어와 빈도를 가지고 판다스 데이터프레임(엑셀 표와 비슷) 생성
adj_df = pd.DataFrame({'형용사':형용사, '형용사빈도':형용사빈도})
# 명사 데이터프레임을 파일로 저장. .xlsx, .csv, .txt 등 가능.
adj_df.to_csv("../dataset/2022/본문_형태소분석/10월/나무문화_10월_형태소분석(형용사).csv", index=False, encoding='UTF-8')
adj_df

1874


,형용사,형용사빈도
0,있다,493555
1,좋다,91001
2,없다,87506
3,같다,77836
4,이다,64995
...,...,...
1869,부듯하다,1
1870,움퍽하다,1
1871,헐렁헐렁하다,1
1872,고실고실하다,1


In [9]:
''' 명사, 동사, 형용사, 조사 똑같이 아래 코드 반복 '''
# 명사 리스트에서 명사 빈도 리스트 생성
print(len(Counter(josas))) # 이 값을 밑에 most_common 안에 넣으면 됨.
josa_cnt = Counter(josas).most_common()

조사 = []
조사빈도 = []
for a, b in josa_cnt:
    조사.append(a)
    조사빈도.append(b)

# 단어와 빈도를 가지고 판다스 데이터프레임(엑셀 표와 비슷) 생성
josa_df = pd.DataFrame({'조사':조사, '조사빈도':조사빈도})
# 명사 데이터프레임을 파일로 저장. .xlsx, .csv, .txt 등 가능.
josa_df.to_csv("../dataset/2022/본문_형태소분석/10월/나무문화_10월_형태소분석(조사).csv", index=False, encoding='UTF-8')
josa_df

400


,조사,조사빈도
0,을,825026
1,의,661771
2,이,638915
3,에,604996
4,를,405067
...,...,...
395,에게서라도,1
396,으로서든지,1
397,이라야만,1
398,와만은,1


In [10]:
df_concat = pd.concat([noun_df,verb_df,adj_df],axis=1)
df_concat.to_csv("../dataset/2022/본문_형태소분석/나무문화_10월_단어종합.txt", index=False, encoding='UTF-8')

In [11]:
df["morphs"] = morphs
df["pos_tag"] = pos_list
df

,split_str,org_idx,morphs,pos_tag
0,10월 1일 날 갔던 캠핑인데 포스팅을 미뤘더니 깜빡해버려서 몰아서 쓰는 중이다,0,"[10월, 1일, 날, 갔던, 캠핑, 인데, 포스팅, 을, 미뤘더니, 깜빡, 해, ...","[(10월, Number), (1일, Number), (날, Noun), (가다, ..."
1,이날 날씨가 너무 좋다 보니 캠핑장과 펜션은 당연히 꽉 차버렸고 캠핑장 사장님 찬스...,0,"[이, 날, 날씨, 가, 너무, 좋다, 보니, 캠핑장, 과, 펜션, 은, 당연히, ...","[(이, Determiner), (날, Noun), (날씨, Noun), (가, J..."
2,친한 언니들 A언니와 B언니 부부와 함께 갔던 날 우리 부부까지 합치면 다섯 명 이...,0,"[친한, 언니, 들, A, 언니, 와, B, 언니, 부부, 와, 함께, 갔던, 날,...","[(친하다, Adjective), (언니, Noun), (들, Suffix), (A..."
3,푸릇푸릇 한 밤나무들이 인상적이다 주차장에도 차가 정 말 많았다,0,"[푸릇푸릇, 한, 밤나무, 들, 이, 인상, 적, 이다, 주차장, 에도, 차가, 정...","[(푸릇푸릇, Noun), (하다, Verb), (밤나무, Noun), (들, Su..."
4,차가 많이 막혀서 일찍 출발했는데도 시간이 꽤 걸렸다,0,"[차가, 많이, 막혀서, 일찍, 출발, 했는데도, 시간, 이, 꽤, 걸렸다]","[(차갑다, Adjective), (많이, Adverb), (막히다, Verb), ..."
...,...,...,...,...
1113264,관람을 마치고 나오면 야외에도 이렇게 좌불상들이 있다,24375,"[관람, 을, 마치, 고, 나오면, 야외, 에도, 이렇게, 좌, 불상, 들, 이, 있다]","[(관람, Noun), (을, Josa), (마치, Noun), (고, Josa),..."
1113265,진짜 중국 스케일 최고 불교는 아니지만 중국 도시를 여행 할 때마다 사찰과 같은 불...,24375,"[진짜, 중국, 스케일, 최고, 불교, 는, 아니지만, 중국, 도시, 를, 여행, ...","[(진짜, Noun), (중국, Noun), (스케일, Noun), (최고, Nou..."
1113266,뭔가 현실세계가 아닌 느낌이랄까,24375,"[뭔가, 현, 실세, 계, 가, 아닌, 느낌, 이, 랄, 까]","[(뭔가, Noun), (현, Modifier), (실세, Noun), (계, Su..."
1113267,그래서 세계인이 주목하는 미래형 불교사찰인거겠지 이정도는 되어야 중국 3대 불교 성...,24375,"[그래서, 세계, 인, 이, 주목, 하는, 미래, 형, 불교, 사찰, 인거, 겠지,...","[(그래서, Adverb), (세계, Noun), (인, Suffix), (이, J..."


In [12]:
df.to_csv("../dataset/2022/본문_형태소분석/나무문화_본문_10월_형태소분석.txt", index=False, encoding='UTF-8')